# Hyperparameter Tuning

In [ ]:
#Basic libraries
import os
import time
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


#Deep Learning libraries
import keras
from keras.layers import Dense, Activation,Dropout, BatchNormalization,LSTM
from keras.models import Sequential
from keras.optimizers import Adam
from keras.activations import sigmoid
from keras.losses import binary_crossentropy
import talos as ta


#Machine Learning libraries
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Define a function to tune each model

For each model, a function will be defined to perform hyperparameter tunning. Parameter grid will be within the function even though it could also be an input if we wanted to modify it for each variable. Inputs will be the data and the name of the variable used for model output. For Machine Learning models `GridSearchCV` will be used, which check every parameter combination on the list using cross-validation. For Deep Learning models `Talos` will be used, which does the same thing but without cross-validation. The function saved the results including variable name so it can be distinguished later.

## Random Forest
For Machine Learning models we just need to initialise the model and then use `GridSearchCV` from `sklearn` with the desired grid. `X` and `y` data are not separated by test and train as cross-validation will be performed.

In [ ]:
def tune_rf(x,y,variable):


    # Create the parameter grid
    rf_grid = {
        'bootstrap': [True],
        'max_depth': [10,30],
        'max_features': ['auto'],
        'min_samples_leaf': [2, 4],
        'min_samples_split': [5,10],
        'n_estimators': list(range(100,2001,100))
    }


    # Create a base model
    rf = RandomForestClassifier(random_state = 42)


    # Instantiate the grid search model (cross-validation=4)
    rf_search = GridSearchCV(rf, rf_grid, cv = 4, n_jobs = -1, verbose = 2)


    # Fit the grid search to the data
    rf_search.fit(x,y)
    
    
    # Save results
    rf_results=pd.DataFrame(rf_search.cv_results_)
    rf_results.drop(['params'],axis=1,inplace=True)
    rf_results.to_csv('hyperparameter\\' + variable +'\\RF_results.csv',index=False)

## k Nearest Neighbors

In [ ]:
def tune_knn(x,y,variable):


    # Create the parameter grid
    knn_grid = {'n_neighbors': list(range(100,1001,20)),
           'weights': ['uniform', 'distance'],
           'leaf_size':[5,30],
           'algorithm':['auto', 'ball_tree']}


    # Create a base model
    knn = KNeighborsClassifier()


    # Instantiate the grid search model (cross-validation=4)
    knn_search = GridSearchCV(knn, knn_grid,  cv = 4, verbose=2, n_jobs = -1)


    # Fit the grid search to the data
    knn_search.fit(x,y)
    
    
    # Save results
    knn_results=pd.DataFrame(knn_search.cv_results_)
    knn_results.drop(['params'],axis=1,inplace=True)
    knn_results.to_csv('hyperparameter\\' + variable + '\\kNN_results.csv',index=False)

## Support Vector Classifier

In [ ]:
def tune_svc(x,y,variable):


    # Create the parameter grid
    svc_grid = {'C': [0.1,1, 10, 100, 1000], 
                  'gamma': ['auto','scale'],
                  'kernel': ['linear','rbf', 'poly', 'sigmoid']}


    # Create a base model
    svc= SVC()


    # Instantiate the grid search model (cross-validation=4)
    svc_search = GridSearchCV(svc,svc_grid,cv=4,verbose=2,n_jobs=-1)


    # Fit the grid search to the data
    svc_search.fit(x,y)
    
    
    # Save results
    svc_results=pd.DataFrame(svc_search.cv_results_)
    svc_results.drop(['params'],axis=1,inplace=True)
    svc_results.to_csv('hyperparameter\\' + variable + '\\SVC_results.csv',index=False)

## Neural Network
For Deep Learning models `Talos` works quite different as we have to manually define the model according to the parameters. After the experiment is run we save the model using the `Deploy` method, which saved the results as well as the best model and its weights in `h5` format, which can be later used.

In [ ]:
#define model to be optimized
def NN_model(x_train, y_train, x_val, y_val, params):

    
    #initialise model
    model = Sequential()
    
    
    #add hidden layers (dense + dropout)
    for i in range(params['hidden_layers']):
        
        
        #first layer must have input dimension according to data
        if i==0:
            model.add(Dense(params['num_neurons'], input_dim=x_train.shape[1],
                            activation=sigmoid,
                            kernel_initializer='normal'))

            model.add(Dropout(params['dropout']))
        
        
        #rest of hidden layers
        else:
            model.add(Dense(params['num_neurons'],
                            activation=sigmoid,
                            kernel_initializer='normal'))

            model.add(Dropout(params['dropout']))
   

    #output layer
    model.add(Dense(1, activation=sigmoid,
                    kernel_initializer='normal'))
    
    
    #compile model
    model.compile(loss=binary_crossentropy,
                  #add a regulizer normalization function from Talos
                  optimizer=Adam(lr=ta.utils.lr_normalizer(params['lr'],Adam)),
                  metrics=['acc'])
    
    
    #fit model
    history = model.fit(x_train, y_train, 
                        validation_data=[x_val, y_val],
                        batch_size=params['batch_size'],
                        epochs=params['epochs'],
                        verbose=0)
    
    #output history and model
    return history, model

In [ ]:
def tune_nn(train_X,train_y,test_X,test_y,variable):


    #create parameter grid
    nn_grid = {'lr': [3.5,5],
         'num_neurons':[50,60,70,80,90,100],
         'hidden_layers':[2,3],
         'batch_size': [16,32,64],
         'epochs': [10,50],
         'dropout': [0.1,0.2]}
    
    
    #create scan object and search (no cross validation)
    nn_search = ta.Scan(x=train_X,
                y=train_y,
                model=NN_model,
                x_val=test_X,
                y_val=test_y,
                params=nn_grid,
                experiment_name='hyper_parameter_NN_' + variable)
    
    
    #save results
    ta.Deploy(scan_object=nn_search, model_name= variable + '_NN_results', metric='val_acc')

## LSTM
For LSTM data must be reshaped as for a 45s input sequence there will be only one output. The function `to_LSTM` does that by grouping X data so that each register has 45points (1 patient), and then reshaping it so that it is (number of patients x 45s x 77 variables) while simplifying output to 1 or 0.

In [ ]:
#fuction to reshape data to match LSTM requirements
def to_LSTM(x_data, y_data):
    
    
    #initialise X and y
    out=[]
    inp=[]
    
    
    #iterate over patients (45s/patient)
    for i in range(int(x_data.shape[0]/45)):
        
        
        #reshape X (per patient)
        X=(pd.DataFrame(x_data).iloc[45*i:45*(i+1),]).values
        inp.append(X.reshape((X.shape[0], 1,X.shape[1])))
        
        
        #turn y to 1 point per patient
        if (pd.DataFrame(y_data).iloc[i*45:(i+1)*45]==1).any()[0]:
            out.append(1)
        else:
            out.append(0)
    
    
    #reshape X (overall)
    inp=np.array(inp)
    inp=inp.reshape((inp.shape[0],inp.shape[1],inp.shape[3]))
    
    
    #output X and y
    return inp,out

In [ ]:
#define model to be optimized
def LSTM_model(x_train, y_train, x_val, y_val, params):

    
    #initialise model
    model = Sequential()
    
    
    #add lstm layer if there must be only one (no return_sequences)
    if params['LSTM_layers']==1:
        model.add(LSTM(params['num_neurons'], input_shape=(x_train.shape[1], x_train.shape[2]),
                           activation=sigmoid,kernel_initializer='normal',dropout=params['dropout']))
        
        
    #if there are many layers (return_sequences needed)
    else:
        
        for i in range(params['LSTM_layers']):
            
            
            #if it is the first layer add input shape
            if i==0:
                model.add(LSTM(params['num_neurons'], input_shape=(x_train.shape[1], x_train.shape[2]),
                           activation=sigmoid,kernel_initializer='normal',dropout=params['dropout'],
                              return_sequences=True))
                
            
            #if it is the last layer return_sequences is not needed
            elif (i+1)==params['LSTM_layers']:
                model.add(LSTM(params['num_neurons'],activation=sigmoid,kernel_initializer='normal',
                          dropout=params['dropout']))
                
                
            #any other layer
            else:
                model.add(LSTM(params['num_neurons'],activation=sigmoid,kernel_initializer='normal',
                          dropout=params['dropout'],return_sequences=True))
            
            
    #add output layer
    model.add(Dense(1, activation=sigmoid,
                    kernel_initializer='normal'))
    
    
    #compile model
    model.compile(loss=binary_crossentropy,
                  #add a regulizer normalization function from Talos
                  optimizer=Adam(lr=ta.utils.lr_normalizer(params['lr'],Adam)),
                  metrics=['acc'])
    
    
    #fit model and save history
    history = model.fit(x_train, y_train, 
                        validation_data=[x_val, y_val],
                        batch_size=params['batch_size'],
                        epochs=params['epochs'],
                        verbose=0)
    
    
    #output history and model
    return history, model

In [ ]:
def tune_lstm(train_X_LSTM,train_y_LSTM,test_X_LSTM,test_y_LSTM,variable):


    #create parameter grid
    lstm_grid = {'lr': [3.5,5],
         'num_neurons':[25,35,45,55,65,75],
         'LSTM_layers':[1,2],
         'batch_size': [16,32,64],
         'epochs': [200,300],
         'dropout': [0.1,0.2]}
    
    
    #create scan object and search (no cross validation)
    lstm_search = ta.Scan(x=train_X_LSTM,
                y=train_y_LSTM,
                model=LSTM_model,
                x_val=test_X_LSTM,
                y_val=test_y_LSTM,
                params=lstm_grid,
                experiment_name='hyper_parameter_LSTM_'+variable)
    
    
    #save results
    ta.Deploy(scan_object=lstm_search, model_name= variable + '_LSTM_results', metric='val_acc')

# Define a function to tune all models
This function works as a `Main()`, it uses the data and variable name as inputs and passes it through tunning functions for each model. It basically tunes every model for given data and stores the results.

In [ ]:
def tune_all(train_X,train_y,test_X,test_y,variable):
    
    
    #merge train and test as for ML methods, cross validation will be used
    x=np.vstack((train_X,test_X))
    y=np.concatenate((train_y,test_y))
    
    
    #ML models
    tune_rf(x,y,variable)
    tune_knn(x,y,variable)
    tune_svc(x,y,variable)
    
    
    #rehsape X and y data for LSTM
    train_X_LSTM,train_y_LSTM=to_LSTM(train_X,train_y)
    test_X_LSTM,test_y_LSTM=to_LSTM(test_X,test_y)
    
    
    #DL models
    tune_nn(train_X,train_y,test_X,test_y,variable)
    tune_lstm(train_X_LSTM,train_y_LSTM,test_X_LSTM,test_y_LSTM,variable)

# Tune all models for each variable
We can use `tune_all` to perform hyperparameter tunning for each variable. First we load the data, the use it as input to run the experiment.

In [ ]:
#train data
train_X_bis=pd.read_csv('data\Data_model\\train_X_BIS.csv',header=None,index_col=False).values
train_X_mov=pd.read_csv('data\Data_model\\train_X_MOV.csv',header=None,index_col=False).values
train_X_nibp=pd.read_csv('data\Data_model\\train_X_NIBP.csv',header=None,index_col=False).values

train_y_bis=pd.read_csv('data\Data_model\\train_y_BIS.csv',header=None,index_col=False)[0].tolist()
train_y_mov=pd.read_csv('data\Data_model\\train_y_MOV.csv',header=None,index_col=False)[0].tolist()
train_y_nibp=pd.read_csv('data\Data_model\\train_y_NIBP.csv',header=None,index_col=False)[0].tolist()


#test data
test_X_bis=pd.read_csv('data\Data_model\\test_X_BIS.csv',header=None,index_col=False).values
test_X_mov=pd.read_csv('data\Data_model\\test_X_MOV.csv',header=None,index_col=False).values
test_X_nibp=pd.read_csv('data\Data_model\\test_X_NIBP.csv',header=None,index_col=False).values

test_y_bis=pd.read_csv('data\Data_model\\test_y_BIS.csv',header=None,index_col=False)[0].tolist()
test_y_mov=pd.read_csv('data\Data_model\\test_y_MOV.csv',header=None,index_col=False)[0].tolist()
test_y_nibp=pd.read_csv('data\Data_model\\test_y_NIBP.csv',header=None,index_col=False)[0].tolist()

In [ ]:
start_time=time.time()

#BIS
tune_all(train_X_bis,train_y_bis,test_X_bis,test_y_bis,'BIS')


#MOV
tune_all(train_X_mov,train_y_mov,test_X_mov,test_y_mov,'MOV')


#NIBP
tune_all(train_X_nibp,train_y_nibp,test_X_nibp,test_y_nibp,'NIBP')

print("--- %s minutes ---" % round((time.time() - start_time)/60))